In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import re
import shapely
import shapely.plotting

In [ ]:

#You'll need to download the geopkg version of the Natural Earth dataset:
# https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/physical/50m_physical.zip

land=gpd.read_file('/Volumes/ice1/ben/NaturalEarth/packages/natural_earth_vector.gpkg', layer = 'ne_50m_land')

In [ ]:
GL_clip = gpd.read_file('assets/GL_offpointing.kml')
GL_clip.geometry[0]
land_GL = land.clip(GL_clip.geometry[0]).set_crs(4326).to_crs(3857)

In [ ]:
plt.figure()
for geom in land_GL.geometry:
    shapely.plotting.plot_polygon(geom, add_points=False)

In [ ]:
land_s=land.cx[:, :-60]

In [ ]:
land_s

In [ ]:
land_s_3857=land_s.to_crs(3857)

In [ ]:
plt.figure()
for jj in land_s.geometry:
    plt.plot(*np.array(jj.exterior.coords).T, marker='.')

In [ ]:
land_AA = shapely.unary_union(land_s_3857.geometry)

In [ ]:
shelf_AA = shapely.unary_union(gpd.read_file('/Volumes/ice1/ben/NaturalEarth/packages/natural_earth_vector.gpkg', layer = 'ne_50m_antarctic_ice_shelves_polys').to_crs(3857).geometry)

In [ ]:
shelf_AA

In [ ]:
import pyproj
bds = [np.array([-179.9999, 179.9999]),  np.array([-89, -60])]

to_wm = pyproj.Transformer.from_crs(4326, 3857)
clip_poly = shapely.Polygon(np.c_[to_wm.transform(*np.c_[bds[1][[0, 0, 1, 1, 0]], bds[0][[0, 1, 1, 0, 0]]].T)])

In [ ]:

AA=shapely.unary_union([land_AA, shelf_AA]).buffer(1000)

In [ ]:
for dist, name in zip([10e3, 50e3, 100e3, 200e3, 500e3, 1000e3], ['10km','50km','100km', '200km', '500km','1000km']):

    AA_buf = AA.buffer(-20000).buffer(20000).buffer(dist).intersection(clip_poly)
    temp=gpd.GeoDataFrame({'name':['Antarctica'], 'geometry':[AA_buf]}).set_crs(3857).to_crs(4326)
    with open(f'assets/Antarctica_{name}.geojson','w') as fh:
        fh.write(temp.to_json())

In [ ]:
! ls assets

In [ ]:
test=gpd.read_file('assets/Antarctica_10km.geojson')

In [ ]:
plt.figure()
colors=['r','orange','gold','green','blue','magenta']
count=0
for dist, name, color in zip([10e3, 50e3, 100e3, 200e3, 500e3, 1000e3], ['10km','50km','100km', '200km', '500km','1000km'], colors):
    print("*"*50)
    print((dist/1000, name))
    AA_buf = AA.buffer(-20000).buffer(20000).buffer(dist).intersection(clip_poly)
    print(AA_buf.area)
    GL_buf = land_GL.buffer(dist)
    geom_dict ={'name':['Antarctica','Greenland'],
                'geometry':[AA_buf, shapely.MultiPolygon(GL_buf.geometry)]}
    temp=None
    temp=gpd.GeoDataFrame(geom_dict).set_crs(3857).to_crs(4326)
    print(temp.geometry[0].area)
    out_file=f'/Antarctica_Greenland_{name}.geojson'
    label=out_file
    if hasattr(AA_buf, 'geoms'):
        for geom in AA_buf.geoms:
            plt.plot(*np.array(geom.exterior.coords).T,marker='.', label=label, color=color)
            label=None
    else:
        plt.plot(*np.array(AA_buf.exterior.coords).T,marker='.', label=out_file, color=color)
    print(out_file)
    with open(f'assets/'+out_file,'w') as fh:
        fh.write(temp.to_json())
plt.legend()

In [ ]:
np.diff(plt.gca().get_ylim())/1000